<a href="https://colab.research.google.com/github/HisakaKoji/bert-japanese/blob/master/notebook/finetune_to_livedoor_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

**This colab notebook assumes the above models are stored on some GSC bucket you can acess its objects.**

In [0]:
%cd /content

/content


In [0]:
%rm -rf bert-japanese

In [0]:
!git clone  https://github.com/HisakaKoji/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 148 (delta 7), reused 11 (delta 3), pack-reused 126
Receiving objects: 100% (148/148), 220.94 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [0]:
%cd /content/bert-japanese/

/content/bert-japanese


In [0]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.82 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [0]:
%cd ..

/content


In [0]:
!pip install -q -r bert-japanese/requirements.txt

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!gsutil cp gs://hisaka/model/wiki-ja.model /content/bert-japanese/model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab /content/bert-japanese/model/

Copying gs://hisaka/model/wiki-ja.model...
/ [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
/ [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [0]:
%cd /content/bert-japanese/notebook

/content/bert-japanese/notebook


In [0]:
%ls

check-extract-features.ipynb   finetune-to-livedoor-corpus.ipynb
check-trained-tokenizer.ipynb  pretraining.ipynb


Check TPU.

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.124.25.66:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3737048413475733321),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9968169853078432389),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 430901788302994980),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 709494376541634037),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1920462314763928166),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13016199878670608326),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 11739130516311576058),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8112092300802901733),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 4773224122509115

## Data preparation

You need to put preprocessed data on your GCS bucket.  
To create preprocessed data, follow https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb.

## Finetune pre-trained model

In [0]:
PRETRAINED_MODEL_PATH = 'gs://hisaka/model/model.ckpt-1400000'  # GCS bucket
INPUT_DATA_GCS = 'gs://hisaka/rurubu'  # GCS bucket
FINETUNE_OUTPUT_DIR = 'gs://hisaka/rurubu/output' # GCS bucket

In [0]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=true \
  --do_eval=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=10.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}

W0828 07:42:33.102778 140013217052544 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0828 07:42:33.105735 140013217052544 deprecation_wrapper.py:119] From ../src/run_classifier.py:858: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0828 07:42:33.106374 140013217052544 deprecation_wrapper.py:119] From ../src/run_classifier.py:663: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0828 07:42:33.106503 140013217052544 deprecation_wrapper.py:119] From ../src/run_classifier.py:663: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0828 07:42:33.106885 140013217052544 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0828 07:

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [0]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=false \
  --do_eval=false \
  --do_predict=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}

W0828 07:44:14.993865 140251878299520 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0828 07:44:14.997009 140251878299520 deprecation_wrapper.py:119] From ../src/run_classifier.py:858: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0828 07:44:14.997752 140251878299520 deprecation_wrapper.py:119] From ../src/run_classifier.py:663: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0828 07:44:14.997915 140251878299520 deprecation_wrapper.py:119] From ../src/run_classifier.py:663: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0828 07:44:14.998378 140251878299520 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0828 07:

## Evaluation

Download result and original data.

In [0]:
!gsutil cp {FINETUNE_OUTPUT_DIR}/test_results.tsv .
!gsutil cp {INPUT_DATA_GCS}/train.tsv .
!gsutil cp {INPUT_DATA_GCS}/dev.tsv .
!gsutil cp {INPUT_DATA_GCS}/test.tsv .

Copying gs://hisaka/rurubu/output/test_results.tsv...
/ [1 files][127.1 KiB/127.1 KiB]                                                
Operation completed over 1 objects/127.1 KiB.                                    
Copying gs://hisaka/rurubu/train.tsv...
/ [1 files][382.5 KiB/382.5 KiB]                                                
Operation completed over 1 objects/382.5 KiB.                                    
Copying gs://hisaka/rurubu/dev.tsv...
/ [1 files][126.8 KiB/126.8 KiB]                                                
Operation completed over 1 objects/126.8 KiB.                                    
Copying gs://hisaka/rurubu/test.tsv...
/ [1 files][128.6 KiB/128.6 KiB]                                                
Operation completed over 1 objects/128.6 KiB.                                    


### Trained model

Check accuracy.

In [0]:
import numpy as np
import pandas as pd

In [0]:
import sys
sys.path.append("../src")

from run_classifier import LivedoorProcessor

processor = LivedoorProcessor()
label_list = processor.get_labels()

In [0]:
result = pd.read_csv("./test_results.tsv", sep='\t', header=None)

In [0]:
result.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.052373,0.021417,0.833535,0.005609,0.003961,0.005431,0.009859,0.021627,0.006582,0.003966,0.001868,0.001484,0.002977,0.000803,0.000990,0.001807,0.001723,0.000982,0.000834,0.000828,0.000832,0.001310,0.000648,0.001049,0.001252,0.003784,0.001659,0.003141,0.006987,0.000681
1,0.593637,0.012734,0.105572,0.008785,0.009416,0.012685,0.027248,0.130120,0.018779,0.008833,0.008165,0.002721,0.008923,0.001346,0.001611,0.003647,0.002625,0.001287,0.001639,0.001537,0.001493,0.002509,0.001127,0.001011,0.001646,0.013223,0.003490,0.002919,0.010230,0.001042


In [0]:
test_df = pd.read_csv("./test.tsv", sep='\t')

In [0]:
test_df['predict'] = [ label_list[np.array(elem[1]).argmax()] for elem in result.iterrows() ]

In [0]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.5806451612903226

In [0]:
### 1/5 of full training data.
# sum( test_df['label'] == test_df['predict'] ) / len(test_df)

In [0]:
test_df.to_csv('./test20190828.csv')

A littel more detailed check using `sklearn.metrics`.

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
print(classification_report(test_df['label'], test_df['predict']))

                                        precision    recall  f1-score   support

                   Agricultural ritual       0.00      0.00      0.00         2
                 Procession-and-parade       0.00      0.00      0.00         9
                    Shrine-floats-etc.       0.00      0.00      0.00        44
Traditional-Festivalsand-annual-events       0.39      0.76      0.52        51
 Traditional-performing-arts-and-dance       0.58      0.74      0.65        62
                                animal       0.00      0.00      0.00         1
                              festival       0.00      0.00      0.00         2
                                  fire       0.20      0.06      0.09        35
                             fireworks       0.79      0.98      0.87       108
                         flower-nature       0.11      0.50      0.18         2
                                  food       0.00      0.00      0.00         7
                          illumination 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
### 1/5 of full training data.
# print(classification_report(test_df['label'], test_df['predict']))

In [0]:
print(confusion_matrix(test_df['label'], test_df['predict']))

[[ 0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  1  3  3  0  0  0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0 19 14  8  0  0  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  3 39  2  0  0  4  2  1  0  0  0  0  0  0  0]
 [ 0  0  6 13 37  0  0  4  1  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0]
 [ 0  2  5  8  1  0  0 17  1  1  0  0  0  0  0  0  0]
 [ 0  0  3  3  0  0  1  2 93  2  0  2  1  0  1  0  0]
 [ 0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  1  1  0  1  1  0  1  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  1  0  0  2  1  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_df['label'], test_df['predict']))

### Simple baseline model.

In [0]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
train_df = pd.read_csv("./train.tsv", sep='\t')
dev_df = pd.read_csv("./dev.tsv", sep='\t')
test_df = pd.read_csv("./test.tsv", sep='\t')

In [0]:
!apt-get -q install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libmecab2 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 14 not upgraded.
Need to get 12.8 MB of archives.
After this operation, 60.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-ipadic all 2.7.0-20070801+main-1 [12.1 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab amd64 0.996-5 [132 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-

In [0]:
!pip install -q mecab-python3==0.7

     |████████████████████████████████| 51kB 2.4MB/s 


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_dev_df = pd.concat([train_df, dev_df])

In [0]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [0]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [0]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(train_df)/len(dev_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

In [0]:
print(classification_report(test_ys, model.predict(test_xs_)))

In [0]:
### 1/5 of full training data.
# print(classification_report(test_ys, model.predict(test_xs_)))

                precision    recall  f1-score   support

dokujo-tsushin       0.82      0.71      0.76       178
  it-life-hack       0.86      0.88      0.87       172
 kaden-channel       0.91      0.87      0.89       176
livedoor-homme       0.67      0.63      0.65        95
   movie-enter       0.87      0.95      0.91       158
        peachy       0.70      0.78      0.73       174
          smax       1.00      1.00      1.00       167
  sports-watch       0.87      0.95      0.91       190
    topic-news       0.92      0.82      0.87       163

     micro avg       0.85      0.85      0.85      1473
     macro avg       0.85      0.84      0.84      1473
  weighted avg       0.86      0.85      0.85      1473



In [0]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[153   4   1   4   2  13   0   1   0]
 [  3 155   6   3   0   1   0   2   2]
 [  0   5 165   0   2   0   1   1   2]
 [  3   4   6  70   4   6   0   1   1]
 [  1   0   1   3 152   1   0   0   0]
 [  7   1   1   2   3 160   0   0   0]
 [  0   0   0   0   0   0 167   0   0]
 [  1   0   0   2   0   0   0 186   1]
 [  3   1   3   5   0   3   0   8 140]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[127   6   1   4   5  29   0   5   1]
 [  2 152   6   7   1   2   0   1   1]
 [  1   8 153   4   5   3   0   1   1]
 [  3   7   3  60   3  15   0   2   2]
 [  1   0   1   1 150   5   0   0   0]
 [ 15   3   0  10   8 135   0   2   1]
 [  0   0   0   0   0   0 167   0   0]
 [  1   0   2   0   0   1   0 181   5]
 [  5   0   2   3   0   4   0  15 134]]
